# Population Density of Mexican States and Municipalities

This Notebook downloads Geopandas GeoDataFrames for States (admin1) and Municipalities (admin2) derived from the 2020 Mexican Census: [INEGI](https://www.inegi.org.mx/temas/mg/).

For details how these dataframe were created, see the [mexican-boundaries](https://github.com/sbl-sdsc/mexico-boundaries) GitHub project.

In [1]:
from io import BytesIO
from urllib.request import urlopen
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

## Boundaries of Mexican Municipalities

Read boundary polygons for Mexican states from shapefile

In [3]:
admin2_url = 'https://raw.githubusercontent.com/sbl-sdsc/mexico-boundaries/main/data/mexico_admin2.parquet'

In [4]:
resp = urlopen(admin2_url)
admin2 = gpd.read_parquet(BytesIO(resp.read()))

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

Calculate the area of each state (convert area from m^2 to km^2

In [ ]:
admin2.crs

In [ ]:
admin2['CVE_MUNI'] = admin2['CVE_ENT'] + admin2['CVE_MUN']

In [ ]:
admin2.head()

In [ ]:
admin2.plot();

## Map of Population by Municipality

Get week 3 analyzes data files

In [ ]:
var_admin2 = pd.read_csv('../../data/week3analyzesMunicipalities.csv')

In [ ]:
var_admin2.head()

Add 5-digit municipality code column (example: convert 5035 -> 05035)

In [ ]:
var_admin2['CVE_MUNI'] = var_admin2['cve_ent'].apply(lambda i: f'{i:05d}')

In [ ]:
var_admin2.head()

Merge the geo dataframe with the population dataframe using the common CVE_MUNI column

In [ ]:
df_admin2 = admin2.merge(var_admin2, on='CVE_MUNI')

In [ ]:
df_admin2.head()

Calculate population density

In [ ]:
a2 = df_admin2.iloc[:,7:].copy()
a2.head()

In [ ]:
a2 = a2[['case_rate', 'death_rate', 'pct_mental_problem', 'pct_no_problems','pct_pop_obesity', 'population/sqkm']].copy()

In [ ]:
#std_scaler = StandardScaler()
std_scaler = RobustScaler()
#std_scaler = MinMaxScaler()
std_scaler
# fit and transform the data
X = pd.DataFrame(std_scaler.fit_transform(a2))

X.head(10)

In [ ]:
# Compute DBSCAN
db = DBSCAN(eps=0.5, min_samples=5).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
#with np.printoptions(threshold=np.inf):
#     print(labels)
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, labels))

In [ ]:
df_labels = pd.DataFrame(labels, columns=['cluster'])

In [ ]:
df2 = pd.concat([df_admin2, df_labels], axis=1)

In [ ]:
df2.head()

In [ ]:
title = 'Population Density Clusters for Municipalities in Mexico'
ax1 = df2.plot(column='cluster', 
#               cmap='OrRd',
# color maps: https://matplotlib.org/stable/tutorials/colors/colormaps.html
               cmap='Set1',
               legend=True, 
               legend_kwds={'label': 'Cluster Number', 
                            'orientation': 'horizontal'},
               figsize=(16, 11));
ax1.set_title(title, fontsize=15);

In [ ]:
# try Plotly with KDE density plot
# bubble maps (see Ebola example with time series):
# https://plotly.com/python/bubble-maps/

Plot population data